In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/scamps_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
phys_net = PhysNet(norm='layer')
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:54630323
Model: "phys_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 128, 32, 32, 16)   1472      
_________________________________________________________________
sequential_1 (Sequential)    (None, 128, 16, 16, 32)   14112     
_________________________________________________________________
sequential_2 (Sequential)    (None, 128, 16, 16, 64)   55616     
_________________________________________________________________
sequential_3 (Sequential)    (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_4 (Sequential)    (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 4, 4, 64)      110720    
__________

In [5]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=1, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet-LN_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet-LN_SCAMPS.h5')

374/374 [==============================] - 224s 582ms/step - loss: 0.5806 - val_loss: 0.3904

Epoch 00001: val_loss improved from inf to 0.39037, saving model to ../weights\PhysNet-LN_SCAMPS.h5


In [4]:
phys_net.load_weights('../weights/PhysNet-LN_SCAMPS.h5')

In [7]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:29<00:00,  2.00it/s]


{'Sliding window': {'MAE': 13.713, 'RMSE': 25.893, 'R': 0.51273},
 'Whole video': {'MAE': 11.765, 'RMSE': 23.306, 'R': 0.59049}}

In [6]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:22<00:00,  1.90it/s]


{'Sliding window': {'MAE': 1.695, 'RMSE': 4.921, 'R': 0.9579},
 'Whole video': {'MAE': 1.237, 'RMSE': 2.391, 'R': 0.9904}}